# Data Validation 3:  Verify that data appears as expected within the MTP UI
2022-10-28 ZD  

## Description
### "Does the MTP UI match the MTP database?"

### Purpose
This validation will test “completeness and accuracy of data loaded into the platform” and follow MTP Data Validations 1&2. It will compare the data displayed within the platform GUI (after loading) to the expected values within the data (before loading). Automated scripts will pull test cases from the data that can be fed into platform testing automations to check displays for completeness and accuracy. 

### Scope
DV3 will focus on displays within the platform that relate to new pediatric data, including those that happen to incorporate Open Targets (OT) data. New pediatric data includes the Food and Drug Administration’s Pediatric Molecular Target Lists (FDA PMTL); the much larger collection of evidence data provided by the Children’s Hospital of Philadelphia (CHoP); and derived summary tables, such as the Pediatric Cancer Data Navigation (PCDN) page. It will not validate or test displays that only include OT data without pediatric data.  

The testing within DV3 will use sampling (spot-checking) methods, though scalability to meet automation capacity will be a design goal. Samples tested will include a set of defined high-profile genes and diseases that we expect will garner an abundance of user attention. We will also include a random sampling of genes and diseases (associated with CHoP data) to expand testing scope.

### Test Case Overview

Testable values for each test case will be contained in a tab within the output Excel

1. Target Association
    - Count of associated diseases
    - PMTL annotation
2. Target Profile Page
    - Somatic Alterations widget
        - Status
        - Row count of each of 5 tabs
    - Gene Expression widget
        - Status
3. Disease Association
    - Count of associated targets
4. Evidence Page
    - Somatic Alterations widget
        - Status
        - Row count of each of 5 tabs
    - Gene Expression widget
        - Status   
5. Pediatric Cancer Data Navigation (PCDN) Page
    - Row count of resulting evidence pages when searching for target or disease
6. Pediatric Molecular Targets List (PMTL) Page
    - Row count of Relevant and Non-Relevant targets  

---

### Updates

**2022-11-15**
- Generate and export new PCDN instead of ingesting
- Add random seed for reproducible random results
- Change Excel export to function with dictionary input for more flexibility

**2022-11-16**
- Update PCDN export functions
    - Export PCDN as partitioned JSONL files
    - Compress files into one folder
    - Create md5 checksum
- Edit output data file structure

## Import modules and define relative paths

In [ ]:
import glob
import json
import os
import random
import hashlib
import shutil

import ndjson
import numpy as np
import pandas as pd

In [ ]:
# CORE VERSIONS
OT_VERSION = '22.04'
OPENPEDCAN_VERSION = 'v11.1'
PMTL_VERSION = 'v3.0'
RANDOM_SEED = 5555

# --------

# INPUTS

# Data from Open Targets
OT_PATH = 'data/external/opentargets/platform/' + OT_VERSION + '/output/etl/json/'

OT_ASSC_OVERALLDIRECT_PATH = OT_PATH + 'associationByOverallDirect/'
OT_ASSC_OVERALLINDIRECT_PATH = OT_PATH + 'associationByOverallIndirect/'
OT_DISEASES_PATH = OT_PATH + 'diseases/'
OT_TARGETS_PATH = OT_PATH + 'targets/'

# Data from CHoP
CHOP_PATH = 'data/raw/OpenPedCan_' + OPENPEDCAN_VERSION + '/'

# CHoP: Somatic Alterations
CNV_PATH = CHOP_PATH + 'gene-level-cnv-consensus-annotated-mut-freq.jsonl.gz'
SNVGENE_PATH = CHOP_PATH + 'gene-level-snv-consensus-annotated-mut-freq.jsonl.gz'
SNV_PATH = CHOP_PATH + 'variant-level-snv-consensus-annotated-mut-freq.jsonl.gz'
FUSIONGENE_PATH = CHOP_PATH + 'putative-oncogene-fused-gene-freq.jsonl.gz'
FUSION_PATH = CHOP_PATH + 'putative-oncogene-fusion-freq.jsonl.gz'

# CHoP: Gene Expression
TPMGENE_PATH = CHOP_PATH + 'long_n_tpm_mean_sd_quantile_gene_wise_zscore.jsonl.gz'
TPMGROUP_PATH = CHOP_PATH + 'long_n_tpm_mean_sd_quantile_group_wise_zscore.jsonl.gz'

# PMTL data
PMTL_PATH = 'data/processed/pmtl_' + PMTL_VERSION + '.json'

# Priority targets and diseases for test cases
PRIORITY_PATH = 'data/processed/dv3_priority_tests/'
PRIORITY_TARGETS_PATH = PRIORITY_PATH + 'targets.csv'
PRIORITY_DISEASES_PATH = PRIORITY_PATH + 'diseases.csv'
PRIORITY_EVIDENCES_PATH = PRIORITY_PATH + 'evidences.csv'

# --------

# OUTPUTS

# Excel file for DV3 automation input
XLSX_OUTPUT = 'dv3_test_cases/MTP_DV3_' + OT_VERSION + '_' + OPENPEDCAN_VERSION + '.xlsx'

# Pediatric Cancer Data Navigation
PCDN_PATH = 'data/processed/pcdn/' + OT_VERSION + '_' + OPENPEDCAN_VERSION + '/'
PCDN_JSON = PCDN_PATH + 'chopDataNavigationTable.json'
PCDN_JSONL = PCDN_PATH + 'jsonl_files'
PCDN_JSONL_TEMP = PCDN_JSONL + '/'
PCDN_JSONL_FILENAMES = 'pcdn_part_{0:04d}.json'

## Load data for processing

In [ ]:
def load_jsonl_files_to_df(path:str, filetype:str='*.json'):
    """
    Load multiple identically-structured jsonl files within a local folder 
    into a single dataframe. Useful for OpenTargets FTP downloads.

    :param path: Relative filepath to the folder containing the jsonl files.
    :param filetype: Filetype suffix of files to include. default '*.json'
    """
    
    # OT uses 'json' extension for 'jsonl' files
    fullPath = path + filetype

    # Create list of all files within path folder
    files = glob.glob(fullPath)

    # Build df by combining all files in path folder
    df = pd.concat(
        (pd.read_json(f, orient='records', lines=True)
        for f in files))

    return df

In [ ]:
# Load OT files as dataframes
ot_asscOverallDirect = load_jsonl_files_to_df(OT_ASSC_OVERALLDIRECT_PATH)
ot_asscOverallIndirect = load_jsonl_files_to_df(OT_ASSC_OVERALLINDIRECT_PATH)
ot_diseases = load_jsonl_files_to_df(OT_DISEASES_PATH)
ot_targets = load_jsonl_files_to_df(OT_TARGETS_PATH)

In [ ]:
# Load CHoP files as dataframes
cnv = pd.read_json(CNV_PATH, orient='records', lines=True)
snvGene = pd.read_json(SNVGENE_PATH, orient='records', lines=True)
snv = pd.read_json(SNV_PATH, orient='records', lines=True)
fusionGene = pd.read_json(FUSIONGENE_PATH, orient='records', lines=True)
fusion = pd.read_json(FUSION_PATH, orient='records', lines=True)

tpmGene = pd.read_json(TPMGENE_PATH, orient='records', lines=True)
tpmGroup = pd.read_json(TPMGROUP_PATH, orient='records', lines=True)

In [ ]:
# Load PMTL as dataframes
pmtl_df = pd.read_json(PMTL_PATH, orient='records')

In [ ]:
# Load list of priorities for testing as dataframes
priority_targets = pd.read_csv(PRIORITY_TARGETS_PATH)
priority_diseases = pd.read_csv(PRIORITY_DISEASES_PATH)
priority_evidences = pd.read_csv(PRIORITY_EVIDENCES_PATH)

## Clean & Transform Data

### Simplify OT Target and Disease datasets

In [ ]:
# Drop all columns except for ids and names
ot_targets = ot_targets.loc[:, ['id', 'approvedSymbol']]
ot_diseases = ot_diseases.loc[:, ['id', 'name']]

### Reformat CHoP TPM files for consistency

In [ ]:
# Define columns to rename
tpmColRenameDict = {
    'Gene_Ensembl_ID': 'targetFromSourceId',
    'EFO': 'diseaseFromSourceMappedId'
}

# Rename columns and add datasourceId column for each file
tpmGene.rename(columns=tpmColRenameDict, inplace=True)
tpmGene['datasourceId'] = 'chop_tpm_genewise_expression'

tpmGroup.rename(columns=tpmColRenameDict, inplace=True)
tpmGroup['datasourceId'] = 'chop_tpm_groupwise_expression'

### Clean IDs within CHoP data

In [ ]:
def clean_chop_targets(df:pd.DataFrame, ot_targets:pd.DataFrame=ot_targets):
    """ Remove rows of evidence that contain blank or incompatible Target IDs.
    These represent values that will not load into the MTP database. 
    
    :param df: Dataframe of CHoP evidence file
    :param ot_targets: Dataframe of Open Targets target database
    """

    # Note any rows with blank target IDs
    blankEvidences = len(df[df['targetFromSourceId'] == ''])
    blankTargets = df[df['targetFromSourceId'] == '']['targetFromSourceId'].nunique()
    print(f"    {blankEvidences} evidences across {blankTargets} blank target IDs removed from {df.datasourceId[0]}")

    # Drop any rows with blank target IDs
    df.drop(df[df['targetFromSourceId'] == ''].index, inplace=True)


    # Enrich chop df with OT target ids and symbols
    df1 = pd.merge(
        df, ot_targets, how='left', left_on='targetFromSourceId', right_on='id').rename(
        columns={'id':'otTargetId', 'approvedSymbol':'otSymbol'})

    # Note any rows with target IDs not found within OT database
    invalidEvidences = len(df1[df1['otTargetId'].isna()])
    invalidTargets = df1[df1['otTargetId'].isna()]['targetFromSourceId'].nunique()
    print(f"    {invalidEvidences} evidences across {invalidTargets} invalid target IDs removed from {df.datasourceId[0]}")

    # Drop any rows with target IDs not found within OT database
    df2 = df1[df1['otTargetId'].notnull()]

    return df2

In [ ]:
def clean_chop_diseases(df:pd.DataFrame, ot_diseases:pd.DataFrame=ot_diseases):
    """ Remove rows of evidence that contain blank or incompatible Disease IDs.
    These represent values that will not load into the MTP database. 
    
    :param df: Dataframe of CHoP evidence file
    :param ot_diseases: Dataframe of Open Targets disease database
    """

    # Note any rows with blank disease IDs
    blankEvidences = len(df[df['diseaseFromSourceMappedId'] == ''])
    blankDiseases = df[df['diseaseFromSourceMappedId'] == '']['diseaseFromSourceMappedId'].nunique()
    print(f"    {blankEvidences} evidences across {blankDiseases} blank disease IDs removed from {df.datasourceId[0]}")

    # Drop any rows with blank disease IDs
    df.drop(df[df['diseaseFromSourceMappedId'] == ''].index, inplace=True)


    # Enrich chop df with OT disease ids and symbols
    df1 = pd.merge(
        df, ot_diseases, how='left', left_on='diseaseFromSourceMappedId', right_on='id').rename(
        columns={'id':'otDiseaseId', 'name':'otDiseaseName'})

    # Note any rows with disease IDs not found within OT database
    invalidEvidences = len(df1[df1['otDiseaseId'].isna()])
    invalidDiseases = df1[df1['otDiseaseId'].isna()]['diseaseFromSourceMappedId'].nunique()
    print(f"    {invalidEvidences} evidences across {invalidDiseases} invalid disease IDs removed from {df.datasourceId[0]}")

    # Drop any rows with disease IDs not found within OT database
    df2 = df1[df1['otDiseaseId'].notnull()]

    return df2


In [ ]:
def chop_cleaning_functions(df:pd.DataFrame, ot_targets:pd.DataFrame=ot_targets, ot_diseases:pd.DataFrame=ot_diseases):
    """ Combines target and disease ID cleaning functions in series. """

    print(df.datasourceId[0], '\n    Start length:', len(df))

    df1 = clean_chop_targets(df)
    df2 = clean_chop_diseases(df1)
    
    print('    End length:', len(df2), '\n---')

    return df2

In [ ]:
# Run each CHoP df through the cleaning functions defined above
cnv_clean = chop_cleaning_functions(cnv)
snvGene_clean = chop_cleaning_functions(snvGene)
snv_clean = chop_cleaning_functions(snv)
fusion_clean = chop_cleaning_functions(fusion)
fusionGene_clean = chop_cleaning_functions(fusionGene)

tpmGene_clean = chop_cleaning_functions(tpmGene)
tpmGroup_clean = chop_cleaning_functions(tpmGroup)

### Build Target-Disease Evidence Dataframe Function
Build a dataframe containing all of the pediatric cancer evidence in the format required for validation. Test cases will be subsets of this df, exported into Excel for automation.

In [ ]:
def build_test_case_df(dfList:list, ot_targets:pd.DataFrame=ot_targets, ot_diseases:pd.DataFrame=ot_diseases):
    """ Build and format a dataframe to use when generating target and evidence page tests.
    Combine and transform a list of cleaned/preprocessed evidence dataframes.
    
    :param dfList: list of pandas DataFrames containing evidence
    :param ot_targets: pandas DataFrame of Open Targets targets to use for reference
    :param ot_diseases: pandas DataFrame of Open Targets diseases to use for reference
    """

    # Create blank output to fill with each evidence df
    dfCombined = pd.DataFrame()

    # Iterate through list of evidence dataframes
    for df in dfList:

        # Group data by 5 columns and get evidence count
        # All evidence must use identical column names/contents
        df1 = df.groupby(
            ['targetFromSourceId', 
            'Gene_symbol', 
            'diseaseFromSourceMappedId', 
            'Disease', 
            'datasourceId']
            ).size().reset_index().rename(columns={0:'evidenceCount'})

        # Add each formatted df into a single dataframe
        dfCombined = pd.concat([dfCombined, df1], ignore_index=True)

    # Pivot to organize datasources as columns showing evidence sums
    df2 = dfCombined.pivot_table(
            values='evidenceCount', 
            index=['targetFromSourceId','diseaseFromSourceMappedId'], 
            columns='datasourceId', 
            aggfunc=sum, fill_value=0
            ).reset_index().rename_axis(None, axis=1)

    # Use target IDs to map canonical OT names for targets
    df3 = pd.merge(df2, ot_targets, how='left', left_on='targetFromSourceId', right_on='id').rename(columns={'approvedSymbol':'targetNameOT'})
    df3.drop(columns='id', inplace=True)

    # Use disease IDs to map canonical OT names for diseases
    df4 = pd.merge(df3, ot_diseases, how='left', left_on='diseaseFromSourceMappedId', right_on='id').rename(columns={'name':'diseaseNameOT'})
    df4.drop(columns='id', inplace=True)

    # Rearrange output columns
    df5 = df4[[
                'targetFromSourceId',
                'diseaseFromSourceMappedId',
                'targetNameOT',
                'diseaseNameOT',
                'chop_gene_level_snv',
                'chop_variant_level_snv',
                'chop_gene_level_cnv',
                'chop_putative_oncogene_fused_gene',
                'chop_putative_oncogene_fusion',
                'chop_tpm_groupwise_expression',
                'chop_tpm_genewise_expression']]

    return df5

### Run function to combine evidence and build test case dataframe

In [ ]:
# Create list of clean dataframes for iteration
dfList = [
    cnv_clean,
    snvGene_clean,
    snv_clean,
    fusion_clean,
    fusionGene_clean,
    tpmGene_clean,
    tpmGroup_clean,
    ]

In [ ]:
# Build test case
testCase_df = build_test_case_df(dfList, ot_targets, ot_diseases)

## Build and export Pediatric Cancer Data Navigation table using the Test Case Dataframe

In [ ]:
def build_pcdn_df(df:pd.DataFrame=testCase_df):
    """ Build the Pediatric Cancer Data Navigation table
    by reformattign the test case dataframe. 
    
    :param df: pandas dataFrame of combined and processed evidence"""

    # Define labels and source columns for True/False summary columns
    # This assumes that gene-level evidence presence matches variant-level
    # presence in relevant data sources
    pcdnSummaryDict = {
    'SNV': 'chop_gene_level_snv',
    'CNV': 'chop_gene_level_cnv',
    'Fusion': 'chop_putative_oncogene_fused_gene',
    'GeneExpression': 'chop_tpm_genewise_expression'
    }

    # Rename columns for clarity and to match input format
    df1 = df.rename(columns={'targetNameOT':'Gene_symbol', 'diseaseNameOT':'Disease'})

    # Create summary True/False columns for evidence presence
    for label, col in pcdnSummaryDict.items():
        df1[label] = np.where(df1[col] > 0, True, False)

    # Keep only columns of interest
    df2 = df1[
        ['targetFromSourceId',
        'diseaseFromSourceMappedId',
        'Gene_symbol',
        'Disease',
        'SNV',
        'CNV',
        'Fusion',
        'GeneExpression']]
    
    return df2

In [ ]:
def export_df_as_json(df:pd.DataFrame, outfile:str):
    """ Export pandas dataframe as JSON file. Parsing using
    json.dumps is needed to avoid extra backslashes otherwise
    present if using only built-in pandas to_json().
    
    :param df: pandas dataframe to export
    :param outfile: filename path for export"""
    
    # Make output directory if it doesn't exist
    if os.path.exists(os.path.dirname(outfile)) == False:
        os.makedirs(os.path.dirname(outfile))

    # Load df as json string and parse
    json_str = df.to_json(orient='records')
    parsed = json.loads(json_str)

    # Export to json filepath
    with open(outfile, 'w') as json_file:
        json_file.write(json.dumps(parsed))

In [ ]:
def export_df_as_jsonl_chunks(df:pd.DataFrame, 
                                directory:str, 
                                maxlines:int, 
                                fileformat:str):
    """ Export pandas dataframe as many JSONL chunks for easier ETL.

    :param df: pandas dataframe to export
    :param directory: filepath of directory to hold exported jsonl
    :param maxlines: int max number of jsonl lines per file
    :param fileformat: name format for export files. 
        Must contain '{}' string for part number formatting
    """

    # Make versioned directory if does not exist
    if os.path.exists(directory) == False:
        os.makedirs(directory)

    # Load pandas df as json lines object
    json_str = df.to_json(orient='records', lines=True)

    # Load json lines as chunks with max lines of 50K per file
    chunks = pd.read_json(json_str, orient='records', lines=True, chunksize=maxlines)

    # Iterate through JSONL partitions to export as files. Show NaN values as blank
    for label, chunk in enumerate(chunks):
        dict_records = chunk.apply(lambda x: x.dropna()).to_dict('records')
        jsonlData = ndjson.dumps(dict_records).encode('utf-8')
        with open(directory+fileformat.format(label), 'wb') as file:
            file.write(jsonlData)

In [ ]:
def output_md5(filename:str, suffix:str='_md5.txt', compression_type:str='zip'):
    """ Create txt file containing md5sum of file. 
    
    :param filename: path of file to compress
    :param suffix: string to attach to end of md5.txt file
    :param compression_type: file extension of filename
    """

    compression_str = '.'+compression_type

    with open(filename+compression_str, 'rb') as f:
        contents = f.read() 
        md5_returned = hashlib.md5(contents).hexdigest()

    with open(filename+suffix, 'w') as file:
        file.write(md5_returned)

In [ ]:
def clear_temp_folder(directory:str):
    """ Delete directory and all contents.

    :param directory: path of directory to delete
    """

    try:
        shutil.rmtree(directory)
    except OSError as error:
        print(f'Error: {directory}: {error.strerror}')

In [ ]:
def compress_directory(filename, root, compression_type='zip'):
    """Compress and zip a file directory.
    
    :param filename: name of the directory to compress. This will
                        also be used as filename for the output
    :param root: parent directory containing the directory to compress
    :param compression_type: compression type string, default 'zip'
    """

    shutil.make_archive(filename, compression_type, root, filename.split('/')[-1])

In [ ]:
def package_jsonl(df:pd.DataFrame,
                    directory:str,
                    maxlines:int,
                    fileformat:str,
                    filename:str,
                    root:str,
                    compression_type:str='zip',
                    suffix:str='_md5.txt'):
    """ Package a pandas dataframe into multiple JSONL files, compress,
    and then get md5 checksum. 
    
    :param df: pandas dataframe to export
    :param directory: filepath of directory to hold exported jsonl
    :param maxlines: int max number of jsonl lines per file
    :param fileformat: name format for export files
                    Must contain '{}' string for part number formatting
    :param filename: name of the directory to compress. This will
                    also be used as filename for the output
    :param root: parent directory containing the directory to compress
    :param compression_type: compression type string, default 'zip'
    :param suffix: string to attach to end of md5.txt file
    """

    export_df_as_jsonl_chunks(df, directory, maxlines, fileformat)
    compress_directory(filename, root, compression_type=compression_type)
    output_md5(filename, suffix=suffix, compression_type=compression_type)
    clear_temp_folder(directory)

In [ ]:
# Buid PCDN
pcdn_df = build_pcdn_df(testCase_df)

# Export as single JSON
export_df_as_json(pcdn_df, PCDN_JSON)

# Export as multiple JSONL
package_jsonl(df=pcdn_df, 
            directory=PCDN_JSONL_TEMP,
            maxlines=3000,
            fileformat=PCDN_JSONL_FILENAMES,
            filename=PCDN_JSONL,
            root=PCDN_PATH,
            compression_type='zip',
            suffix='_md5.txt')

## Build Test Case Functions

### Test Case: Target Associations (Direct Overall)

In [ ]:
def build_testCase_targetAssc(sampleSize:int,
                            ot_asscOverallDirect:pd.DataFrame=ot_asscOverallDirect, 
                            ot_targets:pd.DataFrame=ot_targets, 
                            pmtl_df:pd.DataFrame=pmtl_df, 
                            priority_targets:pd.DataFrame=priority_targets,
                            randomSeed:int=RANDOM_SEED):
    """ Build test case df for Target Associations page. Not specific to new MTP data. 
    Note that until pediatric data is included in association scoring, MTP associations will
    be identical to OT associations.

    :param sampleSize: int number of random targets to include in test
    :param ot_asscOverallDirect: pandas DataFrame of OpenTargets Overall Direct associations
    :param ot_targets: pandas DataFrame of OpenTargets target names for reference
    :param pmtl_df: pandas DataFrame of FDA PMTL
    :param priority_targets: pandas DataFrame of targets to always include in test case
    :param randomSeed: int seed for reproducible random results"""

    # Group associations by targetId to mimic associations heatmap
    df = ot_asscOverallDirect.groupby('targetId').size().reset_index()
    
    # Enrich associations with target names (OT Targets) and PMTL designations
    df1 = df.merge(ot_targets, how='left', left_on='targetId', right_on='id').merge(
                    pmtl_df[['ensemblID', 'designation']], how='left', left_on='targetId', right_on='ensemblID')
    
    # Rename columns
    df2 = df1.rename(columns={
        0:'diseaseCount',
        'approvedSymbol':'targetNameOT',
        'designation':'PMTLcode'})
        
    # Recast NaN PMTL as Unspecified
    df2.fillna('Unspecified Target', inplace=True)

    # Add suffix column
    df2['suffixUrl'] = '/target/'+df2['targetId']+'/associations'

    # Reorder columns and omit redundant
    df3 = df2[['suffixUrl', 'targetId', 'targetNameOT', 'PMTLcode','diseaseCount']]

    # Create subset df with priority targets and random sample of other targets
    random.seed(randomSeed)
    df4 = df3[
            (df3['targetId'].isin(priority_targets['targetId'])) |
            (df3['targetId'].isin(random.sample(df3['targetId'].unique().tolist(), sampleSize)))]

    return df4

### Test Case: Target Profile

In [ ]:
def build_testCase_targetProfile(sampleSize:int, 
                                testCase_df:pd.DataFrame=testCase_df, 
                                priority_targets:pd.DataFrame=priority_targets,
                                randomSeed:int=RANDOM_SEED):
    """ Build test case df for Target Profile page. Should only include targets with
    at least some new pediatric data. 

    :param sampleSize: int number of random targets to include in test case
    :param testCase_df: pandas DataFrame of preprocessed evidence data
    :param priority_targets: pandas DataFrame of targets to always include in test case
    :param randomSeed: int seed for reproducible random results"""

    df = testCase_df.groupby(['targetFromSourceId','targetNameOT']).sum().reset_index()

    # Add TRUE/FALSE for presence of Gene Expression Widget (groupwise plot)
    df['opcGeneExp_target'] = np.where(df['chop_tpm_groupwise_expression'] >0, 'TRUE', 'FALSE')

    # List all datasourceIds to be grouped and considered Somatic Alterations with tab labels
    somaticAltCols = {
    'chop_gene_level_cnv':'cnvByGene',
    'chop_gene_level_snv':'snvByGene',
    'chop_putative_oncogene_fused_gene':'fusionByGene',
    'chop_putative_oncogene_fusion':'fusion',
    'chop_variant_level_snv':'snvByVariant'}

    # Raname columns
    df.rename(columns=somaticAltCols, inplace=True)
    df.rename(columns={'targetFromSourceId':'targetId'}, inplace=True)

    # Add TRUE/FALSE for presence of Somatic Alterations widget
    df['opcSomaticAlt'] = np.where(df[somaticAltCols.values()].sum(axis=1) > 0, 'TRUE', 'FALSE')

    # Add suffix column
    df['suffixUrl'] ='/target/'+df['targetId']

    # Rearrange columns and omit redundant
    df1 = df[['suffixUrl', 'targetId', 'targetNameOT', 'opcGeneExp_target', 'opcSomaticAlt', 'snvByGene', 'snvByVariant', 'cnvByGene','fusionByGene','fusion']]

    # Create subset df with priority targets and  random sample of other targets
    random.seed(randomSeed)
    df2 = df1[
            (df1['targetId'].isin(priority_targets['targetId'])) |
            (df1['targetId'].isin(random.sample(df1['targetId'].unique().tolist(), sampleSize)))]

    return df2

### Test Case: Disease Associations (Indirect Overall)

In [ ]:
def build_testCase_diseaseAssc(sampleSize:int,
                            ot_asscOverallIndirect:pd.DataFrame=ot_asscOverallIndirect, 
                            ot_diseases:pd.DataFrame=ot_diseases,
                            priority_diseases:pd.DataFrame=priority_diseases,
                            randomSeed:int=RANDOM_SEED):
    """ Build test case df for Target Associations page. Not specific to new MTP data. 
    Note that until pediatric data is included in association scoring, MTP associations will
    be identical to OT associations.

    :param sampleSize: int number of random diseases to include in test
    :param ot_asscOverallIndirect: pandas DataFrame of OpenTargets Overall Indirect associations
    :param ot_diseases: pandas DataFrame of OpenTargets disease names for reference
    :param priority_diseases: pandas DataFrame of diseases to always include in test case
    :param randomSeed: int seed for reproducible random results
    """

    # Group associations by targetId to mimic associations heatmap
    df = ot_asscOverallIndirect.groupby('diseaseId').size().reset_index()
    
    # Enrich associations with target names (OT Targets) and PMTL designations
    df1 = df.merge(ot_diseases, how='left', left_on='diseaseId', right_on='id')
    
    # Rename columns
    df2 = df1.rename(columns={
        0:'targetCount',
        'name':'diseaseNameOT'})

    # Add suffix column
    df2['suffixUrl'] = '/disease/'+df2['diseaseId']+'/associations'

    # Reorder columns and omit redundant
    df3 = df2[['suffixUrl', 'diseaseId', 'diseaseNameOT','targetCount']]

    # Create subset df with priority targets and random sample of other targets
    random.seed(randomSeed)
    df4 = df3[
            (df3['diseaseId'].isin(priority_diseases['diseaseId'])) |
            (df3['diseaseId'].isin(random.sample(df3['diseaseId'].unique().tolist(), sampleSize)))]

    return df4

### Test Case: Evidence Page

In [ ]:
def build_testCase_evidence(sampleSize:int, 
                            testCase_df:pd.DataFrame=testCase_df, 
                            priority_evidences:pd.DataFrame=priority_evidences,
                            randomSeed:int=RANDOM_SEED):
    """ Build test case df for Evidence page. Should only include target-disease
    evidence with at least some new pediatric data. 

    :param sampleSize: int number of random evidence combinations to include in test case
    :param testCase_df: pandas DataFrame of preprocessed evidence data
    :param priority_evidences: pandas DataFrame of evudebces to always include in test case
    :param randomSeed: int seed for reproducible random results"""

    # Build subset of testCase df using only priority evidences
    priorityEvidence = pd.merge(priority_evidences[['targetId','diseaseId']], testCase_df, how='left', 
        left_on=['targetId','diseaseId'], right_on=['targetFromSourceId','diseaseFromSourceMappedId'])
    priorityEvidence.drop(columns=['targetId','diseaseId'], inplace=True)

    # Build subset of testCase df using random sample of evidence combinations
    random.seed(randomSeed)
    sampleEvidence = testCase_df.groupby(['targetFromSourceId', 'diseaseFromSourceMappedId']).size().reset_index().sample(sampleSize)
    sampleEvidence.drop(columns=0, inplace=True)
    sampleEvidenceDetails = pd.merge(sampleEvidence, testCase_df, how='left', on=['targetFromSourceId', 'diseaseFromSourceMappedId'])

    # Concat priority and random subset dfs
    df = pd.concat([priorityEvidence, sampleEvidenceDetails], ignore_index=True)

    # Add TRUE/FALSE for presence of Gene Expression Widget (genewise plot)
    df['opcGeneExp_evidence'] = np.where(df['chop_tpm_genewise_expression'] >0, 'TRUE', 'FALSE')

    # List all datasourceIds to be grouped and considered Somatic Alterations with tab labels
    somaticAltCols = {
    'chop_gene_level_cnv':'cnvByGene',
    'chop_gene_level_snv':'snvByGene',
    'chop_putative_oncogene_fused_gene':'fusionByGene',
    'chop_putative_oncogene_fusion':'fusion',
    'chop_variant_level_snv':'snvByVariant'}

    # Rename columns
    df.rename(columns=somaticAltCols, inplace=True)
    df.rename(columns={'targetFromSourceId':'targetId', 'diseaseFromSourceMappedId':'diseaseId'}, inplace=True)

    # Add TRUE/FALSE for presence of Somatic Alterations widget
    df['opcSomaticAlt'] = np.where(df[somaticAltCols.values()].sum(axis=1) > 0, 'TRUE', 'FALSE')

    # Add suffix column
    df['suffixUrl'] ='/evidence/'+df['targetId']+'/'+df['diseaseId']

    # Rearrange columns and omit redundant
    df1 = df[['suffixUrl', 'targetId', 'diseaseId', 'targetNameOT', 'diseaseNameOT', 'opcGeneExp_evidence', 'opcSomaticAlt', 'snvByGene', 'snvByVariant', 'cnvByGene','fusionByGene','fusion']]

    return df1


### Test Case: Pediatric Cancer Data Navigation (PCDN) by Gene

In [ ]:
def build_testCase_pcdnGene(sampleSize:int, 
                        pcdn_df:pd.DataFrame=pcdn_df, 
                        priority_targets:pd.DataFrame=priority_targets,
                        randomSeed:int=RANDOM_SEED):
    """ Build test case df for PCDN page, gene search. Should only include 
    targets contained within the PCDN data (and therefore new pediatric data). 

    :param sampleSize: int number of random targets to include in test case
    :param pcdn_df: pandas DataFrame of preprocessed Pediatric Cancer Data Navigation page
    :param priority_targets: pandas DataFrame of targets to always include in test case
    :param randomSeed: int seed for reproducible random results"""

    # Build subset of the PCDN df using priority targets and random sample of other targets
    random.seed(randomSeed)
    df = pcdn_df[
        (pcdn_df['Gene_symbol'].isin(priority_targets['symbol'])) |
        (pcdn_df['Gene_symbol'].isin(random.sample(pcdn_df['Gene_symbol'].unique().tolist(), sampleSize)))]

    # Group subset by gene symbol to get resulting evidence count
    df1 = df.groupby('Gene_symbol').size().reset_index()

    # Rename columns
    df1.rename(columns={0:'evidenceResults', 'Gene_symbol':'name'}, inplace=True)

    # Add suffix and category columns
    df1['suffixUrl'] = '/pediatric-cancer-data-navigation'
    df1['category'] = 'target'

    # Rearrange columns and omit redundant
    df2 = df1[['suffixUrl', 'category', 'name', 'evidenceResults']]

    return df2

### Test Case: Pediatric Cancer Data Navigation (PCDN) by Disease

In [ ]:
def build_testCase_pcdnDisease(sampleSize:int, 
                        maxResults:int=10000,
                        pcdn_df:pd.DataFrame=pcdn_df, 
                        priority_diseases:pd.DataFrame=priority_diseases,
                        randomSeed:int=RANDOM_SEED):
    """ Build test case df for PCDN page, disease search. Should only include 
    diseases contained within the PCDN data (and therefore new pediatric data). 

    :param sampleSize: int number of random diseases to include in test case
    :param maxResults: int maximum count of evidence results. Default is 10,000 to match MTP
    :param pcdn_df: pandas DataFrame of preprocessed Pediatric Cancer Data Navigation page
    :param priority_diseases: pandas DataFrame of diseases to always include in test case
    :param randomSeed: int seed for reproducible random results"""

    # Build subset of the PCDN df using priority diseases and random sample of other diseases
    random.seed(randomSeed)
    df = pcdn_df[
        (pcdn_df['Disease'].isin(priority_diseases['name'])) |
        (pcdn_df['Disease'].isin(random.sample(pcdn_df['Disease'].unique().tolist(), sampleSize)))]

    # Group subset by gene symbol to get resulting evidence count
    df1 = df.groupby('Disease').size().reset_index().rename(columns={0:'fullResults'})

    # Cap number of results at a maximum (default 10,000) to match site functionality
    df1['evidenceResults'] = np.where(df1['fullResults'] > maxResults, maxResults, df1['fullResults'])

    # Rename columns
    df1.rename(columns={'Disease':'name'}, inplace=True)

    # Add suffix and category columns
    df1['suffixUrl'] = '/pediatric-cancer-data-navigation'
    df1['category'] = 'disease'

    # Rearrange columns and omit redundant
    df2 = df1[['suffixUrl', 'category', 'name', 'evidenceResults']]

    return df2

### Test Case: Pediatric Molecular Targets Lists (PMTL)

In [ ]:
def build_testCase_pmtl(pmtl_df:pd.DataFrame=pmtl_df):
    """ Build test case df for FDA PMTL page.

    :param pmtl_df: pandas DataFrame of the computable FDA PMTL"""

    df = pmtl_df.groupby('designation').size().reset_index()

    df['suffixUrl'] = '/fda-pmtl'
    df['category'] = 'designation'

    df.rename(columns={0:'count', 'designation':'categoryValue'}, inplace=True)

    df1 = df[['suffixUrl', 'category', 'categoryValue', 'count']]

    return df1

## Generate Test Cases

In [ ]:
# Run test case functions to build dataframes for excel export
targetAssc = build_testCase_targetAssc(10)
targetProfile = build_testCase_targetProfile(10)
diseaseAssc = build_testCase_diseaseAssc(10)
evidence = build_testCase_evidence(15)
pcdnGene = build_testCase_pcdnGene(10)
pcdnDisease = build_testCase_pcdnDisease(10)
pmtl = build_testCase_pmtl()

## Export Test Cases to Excel

In [ ]:
# Build df with test generation metadata
sourceData = pd.DataFrame({
    'Open Targets Version':OT_VERSION, 
    'CHoP OpenPedCan Version': OPENPEDCAN_VERSION, 
    'PMTL Version':PMTL_VERSION,
    'Test Case Generation Date':pd.Timestamp.now(),
    'Reproducible Random Seed':RANDOM_SEED
    }.items(),
    columns=['MTP DV3 TEST CASES FOR AUTOMATION', ''])

In [ ]:
# Define and name test case dfs for export
outputDict = {
    'sourceData': sourceData,
    'targetAssc': targetAssc,
    'targetProfile': targetProfile,
    'diseaseAssc': diseaseAssc,
    'evidence': evidence,
    'pcdnGene': pcdnGene,
    'pcdnDisease': pcdnDisease,
    'pmtl': pmtl}

In [ ]:
def export_test_cases_as_excel(outputDict:dict, outfile:str=XLSX_OUTPUT):
    """ Build and format an Excel file containing test cases for automation.
    
    :param outfile: filepath for Excel output file
    :param outputDict: dict of test cases to export as Excel sheets"""

    # Create writer using XlsxWriter as the engine
    writer = pd.ExcelWriter(outfile, engine='xlsxwriter')

    # Write each test case df defined in outputDict to a sheet
    for sheetname, df in outputDict.items():
        df.to_excel(writer, sheet_name=sheetname, index=False)
    
    # Format width of (descriptive) first sheet columns for readability
    writer.sheets[list(outputDict.keys())[0]].set_column(0, 0, 40)
    writer.sheets[list(outputDict.keys())[0]].set_column(1, 1, 20)

    # Save and close excel writer
    writer.save()

In [ ]:
# Run export function
export_test_cases_as_excel(outputDict, XLSX_OUTPUT)